In [1]:
import numpy as np
import pandas as pd
import gzip
from collections import defaultdict
from tqdm import tqdm
from os.path import join
from Bio import SeqIO
import os

In [2]:
from alignment import *

### Set variables

In [3]:
data_path = '/data2/genome/human_GRCh38/ncbi/'
mito_chrom = 'NC_012920.1'

window =150 #bps covered for sequencing


## Load genomic info and extract mitochondria information
- MT fasta sequence
- gff file
- MT genes in fasta

In [4]:
full_gff_filename = join(data_path,'GRCh38_latest_genomic.gff')
gff_filename = "MT.gff"
if not os.path.exists(gff_filename):
    cmd = f"grep {mito_chrom} {full_gff_filename} > {gff_filename}"
    print(cmd)
    os.system(cmd)

gff_info = read_gff(gff_filename, file_format=None)
## Analyzing GFF
gff_info.Type.unique()

gff_info = gff_info.loc[gff_info.Location == mito_chrom].reset_index(drop=True)
gff_info

Reading line 0


,Id,Type,Start,End,Strand,Location
0,id1959760,region,1,16569,+,NC_012920.1
1,gene60922,gene,577,647,+,NC_012920.1
2,rna171173,tRNA,577,647,+,NC_012920.1
3,id1959761,exon,577,647,+,NC_012920.1
4,gene60923,gene,648,1601,+,NC_012920.1
...,...,...,...,...,...,...
96,id1959784,exon,15888,15953,+,NC_012920.1
97,gene60958,gene,15956,16023,-,NC_012920.1
98,rna171196,tRNA,15956,16023,-,NC_012920.1
99,id1959785,exon,15956,16023,-,NC_012920.1


In [5]:
gff_info_genes = gff_info[gff_info["Type"]=='gene']

In [6]:
gff_info_genes["Length"] = gff_info_genes["End"]-gff_info_genes["Start"]+1
gff_info_genes.to_csv("Gene_lengths.csv", index=None)
gff_info_genes

/home/isshamie/software/anaconda2/envs/mito_trace/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Id,Type,Start,End,Strand,Location,Length
1,gene60922,gene,577,647,+,NC_012920.1,71
4,gene60923,gene,648,1601,+,NC_012920.1,954
7,gene60924,gene,1602,1670,+,NC_012920.1,69
10,gene60925,gene,1671,3229,+,NC_012920.1,1559
14,gene60926,gene,3230,3304,+,NC_012920.1,75
17,gene60927,gene,3307,4262,+,NC_012920.1,956
19,gene60928,gene,4263,4331,+,NC_012920.1,69
22,gene60929,gene,4329,4400,-,NC_012920.1,72
25,gene60930,gene,4402,4469,+,NC_012920.1,68
28,gene60931,gene,4470,5511,+,NC_012920.1,1042
